<a href="https://colab.research.google.com/github/nicobrx/prettify_gtm_export/blob/main/prettify_GTM_JSON_export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Create a human-readable Google Doc from a Google Tag Manager JSON export

### How to use this notebook



1.   [Generate an export of a GTM container](https://support.google.com/tagmanager/answer/6106997?hl=en#:~:text=to%20multiple%20containers.-,Export%20a%20container,-In%20the%20top)
2.   Run the code cell below 'Imports and authentication' to import libraries and authenticate using a Google account - this is so you can create a Google Doc
3. Run the next code cell to upload your GTM export file. Click 'Choose Files' to find it on your local drive.
4. Run the next code cell to create an HTML version of the output. You can find the HTML file by clicking on the folder icon in the left bar. Files are not saved permanently in Colab, so you'll want to download it if you want a copy of the HTML version.
5. Run the last code cell to create a Google Doc version.



### Imports and authentication

In [5]:
from google.colab import files
from google.colab import auth
from google.auth import default
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import json

# Authenticate and mount Google Drive
auth.authenticate_user()
creds, _ = default()

### Upload a Google Tag Manager container export file

In [6]:
# upload the GTM JSON export

upload = files.upload()
filename = list(upload.keys())[0]
with open(filename, 'r') as file:
    gtm_data = json.load(file)


Saving GTM-MTTGZMW_workspace229.json to GTM-MTTGZMW_workspace229 (1).json


### Create HTML documentation from export data

In [20]:
# need to fill out complete list of built-in trigger types -
built_in_triggers = {
    '2147479553':'All Pages',
    '2147479572':'Consent Initialization'
}

# the GTM export uses tag type names that can be hard to interpret - these are lookups to make them easier to understand
tag_types = {
  'awct':'Google Ads Conversion Tracking',
  'baut':'Microsoft Advertising Universal Event Tracking',
  'bzi':'LinkedIn Insight',
  'cegg':'Crazy Egg',
  'cvt_31071642_5':'Facebook Pixel',
  'cvt_2172125_189':'Facebook Pixel',
  'cvt_31163855_152':'Lucky Orange Tracking Code',
  'cvt_31163855_149':'OneTrust CMP',
  'cvt_166596669_96':'OneTrust CMP',
  'cvt_2715620_170':'LinkedIn InsightTag 2.0',
  'cvt_8457446_299':'LinkedIn InsightTag 2.0',
  'cvt_2715620_175':'Cookiebot CMP',
  'cvt_6370438_110':'Nextdoor Pixel',
  'flc':'Floodlight Counter',
  'fls':'Floodlight Sales',
  'gaawe':'GA4 Event',
  'gclidw':'Conversion Linker',
  'googtag': 'Google Tag',
  'hjtc':'Hotjar Tracking Code',
  'html':'Custom HTML',
  'img':'Custom Image',
  'pntr':'Pinterest Tag',
  'qpx':'Quora Pixel',
  'sp':'Google Ads Remarketing',
  'ua':'Google Analytics: Universal Analytics'

}

variable_types = {
  'aev':'Auto-Event Variable',
  'awec':'User-Provided Data',
  'c':'Constant',
  'd':'DOM element',
  'gas':'Google Analytics Settings',
  'j':'JavaScript Variable',
  'jsm':'Custom JavaScript',
  'k':'1st Party Cookie',
  'remm':'RegEx Table',
  'smm':'Lookup Table',
  'u':'URL',
  'v':'DataLayer Variable'
}

# Extract tags, triggers, and variables
tags = gtm_data.get('containerVersion',{}).get('tag',[])
triggers = gtm_data.get('containerVersion',{}).get('trigger',[])
variables = gtm_data.get('containerVersion',{}).get('variable',[])

# Extract container name
container_name = gtm_data.get('containerVersion',{}).get('container',{}).get('name','Unknown Container Name')

# Create dictionaries to map trigger IDs to names for easier lookup
trigger_id_to_name = {trigger['triggerId']: trigger['name'] for trigger in triggers}
trigger_id_to_name.update(built_in_triggers)

# Build the HTML content
html_content = "<html><body>"

# Tags
html_content += "<h1>Tags</h1>"
html_content += "<ul>"
for tag in tags:
    tag_name = tag.get('name', 'Unknown Tag Name')
    tag_type = tag.get('type', 'Unknown Type')
    tag_type_name = tag_types.get(tag_type, tag_type)
    tag_params = tag.get('parameter', [])

    # Extract firing triggers using trigger IDs
    firing_triggers = [trigger_id_to_name.get(trigger_id, trigger_id) for trigger_id in tag.get('firingTriggerId', [])]

    html_content += f"<li><strong>{tag_name}</strong> (Type: {tag_type_name})"
    html_content += "<ul>"

    html_content += "<li><strong>Parameters:</strong><ul>"
    for param in tag_params:
        if param.get('key') == 'eventSettingsTable':
          for item in param['list']:
            if 'map' in item:
                param_value = None
                param_value_value = None
                # Loop through the 'map' section of each item
                for entry in item['map']:
                    if entry['key'] == 'parameter':
                        param_value = entry['value']
                    elif entry['key'] == 'parameterValue':
                        param_value_value = entry['value']

                # Add list items to HTML if both 'parameter' and 'parameterValue' are found
                if param_value and param_value_value:
                    html_content += f"<li>Event parameter: {param_value}, Value: {param_value_value}</li>"
        else:
          param_key = param.get('key', 'Unknown Key')
          param_value = param.get('value', 'Unknown Value')
          html_content += f"<li>{param_key}: {param_value}</li>"
    html_content += "</ul></li>"

    html_content += "<li><strong>Firing Triggers:</strong><ul>"
    for trigger_name in firing_triggers:
        html_content += f"<li>{trigger_name}</li>"
    html_content += "</ul></li>"

    html_content += "</ul></li>"
html_content += "</ul>"

# Triggers
html_content += "<h1>Triggers</h1>"
html_content += "<ul>"
for trigger in triggers:
    trigger_name = trigger.get('name', 'Unknown Trigger Name')
    trigger_type = trigger.get('type', 'Unknown Type')
    filters = trigger.get('filter', [])

    html_content += f"<li><strong>{trigger_name}</strong> (Type: {trigger_type})"

    # Check if there are any filters
    if filters:
        html_content += "<ul><li><strong>Filter Conditions:</strong><ul>"
        for f in filters:
            filter_type = f.get('type', 'Unknown Filter Type')
            parameters = f.get('parameter', [])

            # Extract filter conditions
            param_descriptions = {}
            for param in parameters:
                param_key = param.get('key', 'Unknown Key')
                param_value = param.get('value', 'Unknown Value')
                param_descriptions[param_key] = param_value
            param_text = param_descriptions['arg0'] + ' ' + filter_type + ' ' + param_descriptions['arg1']
            html_content += f"<li>{param_text}</li>"
        html_content += "</ul></li></ul>"

    html_content += "</li>"
html_content += "</ul>"

# Variables
html_content += "<h1>Variables</h1>"
html_content += "<ul>"
for variable in variables:
    variable_name = variable.get('name', 'Unknown Variable Name')
    variable_type = variable.get('type', 'Unknown Type')
    variable_type_name = variable_types.get(variable_type, variable_type)

    html_content += f"<li><strong>{variable_name}</strong> (Type: {variable_type_name})</li>"
html_content += "</ul>"

html_content += "</body></html>"

# Save the HTML content to a file
html_file_path = '/content/gtm_container_documentation.html'
with open(html_file_path, 'w') as html_file:
    html_file.write(html_content)

### Create a Google doc from the output

In [21]:
# Create a new Google Doc
docs_service = build('docs', 'v1', credentials=creds)
document = docs_service.documents().create(body={'title': f'{container_name} GTM Container Documentation'}).execute()
doc_id = document['documentId']

# Convert HTML to Google Docs format and upload
drive_service = build('drive', 'v3', credentials=creds)
media = MediaFileUpload(html_file_path, mimetype='text/html', resumable=True)
drive_response = drive_service.files().update(
    fileId=doc_id,
    media_body=media,
    fields='id'
).execute()

doc_link = f"https://docs.google.com/document/d/{doc_id}/edit"

print(f"Google Doc Link: {doc_link}")

Google Doc Link: https://docs.google.com/document/d/1cM5mQXFMZrPC6l9Y8LjlV3nKMWpP8UmEW6qpQqSuoSs/edit
